1. Transcription en Dialecte Tunisien

Modèle ASR Recommandé :

Tunisian_Automatic_Speech_Recognition (SalahZa) 6

Performance : CER 6.22% / WER 10.55% sur TARIC

Avantages :

Traite le code-switching (Arabe/Français/Anglais)

Modèle léger pour terminaux mobiles

In [ ]:
from transformers import pipeline
asr_tn = pipeline("automatic-speech-recognition", model="SalahZa/Tunisian_Automatic_Speech_Recognition")
transcription = asr_tn("audio.wav")["text"]  # Transcription en texte

Alternative Optimisée Mobile :

LinTO ASR 10

Usage :

python
from vosk import Model, KaldiRecognizer
model = Model("linto-asr-ar-tn-0.1/android-model")  # Version allégée
recognizer = KaldiRecognizer(model, 16000)
recognizer.AcceptWaveform(audio_data)
transcription = json.loads(recognizer.FinalResult())["text"]

2. Détection de Stress Vocal (SER)
Stack Technique :

Framework : SpeechBrain 1

Données d'entraînement :

TunSwitch : 2631 phrases tunisiennes étiquetées 315

CREMA-D : 7 442 clips avec 6 émotions (colère, peur, joie, etc.) 4

MediaSpeech : Émotions en Arabe/Français 4

In [ ]:
from speechbrain.pretrained import EncoderClassifier
classifier = EncoderClassifier.from_hparams(
    source="speechbrain/emotion-recognition",
    savedir="pretrained_models/emotion-recognition"
)
# Adaptation avec données tunisiennes
classifier.fit(
    train_data="datasets/TunSwitch/train.csv",
    emotion_labels=["stress", "normal", "crise"],
    epochs=10
)

Extraction des Features Clés :



In [ ]:
import librosa
def extract_stress_features(audio_path):
    y, sr = librosa.load(audio_path)
    f0 = librosa.pyin(y, fmin=80, fmax=400)[0]  # Hauteur vocale
    rms = librosa.feature.rms(y=y)[0]            # Énergie sonore
    return [np.nanmean(f0), np.std(f0), np.mean(rms)]

3. Intégration Temps Réel
   graph TD
    A[Microphone] --> B(Transcription via SalahZa/LinTO)
    A --> C(Détection Stress via SpeechBrain)
    B --> D{Analyse Sémantique}
    C --> E[Score de Stress]
    D --> F[Mots-clés : “aide”, “danger”]
    E & F --> G[Décision Alerte]


In [ ]:
import sounddevice as sd
from scipy.io.wavfile import write

def audio_callback(indata, frames, time, status):
    audio = indata[:, 0]
    # 1. Transcription
    transcript = asr_tn(audio)["text"]
    # 2. Détection stress
    features = extract_stress_features(audio)
    stress_score = classifier.predict(features)
    # 3. Détection mots-clés
    keywords = ["secours", "danger", "aidez-moi"]
    alert = any(kw in transcript for kw in keywords) or stress_score > 0.8
    if alert:
        trigger_emergency(location=gps.get_location(), audio=audio)

puis il reste la conversion en lite